# Hamiltonian Matrix Construction

### Imports

In [3]:
from pyscf import gto, scf
import scipy.sparse as sp
import numpy as np
from scipy.sparse.linalg import eigsh
import scipy.linalg as la
import scipy.sparse as sp

### Tries to construct the hamiltonian matrix

In [13]:
# Define the molecule
mol = gto.M(atom=[
    ['O', (0.0, 0.0, 0.0)],          # Oxygen at origin
    ['H', (0.0, -0.757, 0.587)],     # Hydrogen 1
    ['H', (0.0, 0.757, 0.587)]       # Hydrogen 2
], basis='6-311G')

# Perform Hartree-Fock Calculation
mf = scf.RHF(mol)
mf.kernel()

# Obtain Molecular Orbitals Coefficients and Integrals
mo_coeff = mf.mo_coeff
h1 = mf.get_hcore(mol)
g2 = mol.intor('int2e', aosym='s1')

# Transform Two-Electron Integrals to MO Basis
g2_mo = np.einsum('pqrs,pi,qj,rk,sl->ijkl', g2, mo_coeff, mo_coeff, mo_coeff, mo_coeff)

# Number of orbitals and occupied orbitals
num_orbitals = mo_coeff.shape[1]
nocc = mol.nelectron // 2  # Assuming a closed-shell molecule

# Compute the Density Matrix
P = 2 * np.dot(mo_coeff[:, :nocc], mo_coeff[:, :nocc].T)

# Construct the Fock Matrix
F = np.copy(h1)
for i in range(num_orbitals):
    for j in range(num_orbitals):
        for k in range(num_orbitals):
            for l in range(num_orbitals):
                F[i, j] += P[k, l] * (2 * g2_mo[i, j, k, l] - g2_mo[i, k, j, l])

# Expand the Fock Matrix to include spin orbitals
num_spin_orbitals = 2 * num_orbitals
F_spin = np.zeros((num_spin_orbitals, num_spin_orbitals))
for i in range(num_orbitals):
    for j in range(num_orbitals):
        F_spin[i, j] = F[i, j]
        F_spin[i + num_orbitals, j + num_orbitals] = F[i, j]

# Convert the Fock Matrix to a sparse matrix


# Step 8: Scale Up to a Larger Sparse Matrix
# The Hamiltonian matrix is converted into a sparse matrix format.
H_spin_sparse = sp.csr_matrix(F_spin)

# A much larger sparse matrix (`large_H_spin_sparse`) is created.
size = 2**14
large_H_spin_sparse = sp.lil_matrix((size, size))
# Convert H_spin to a sparse matrix
H_spin_sparse = sp.csr_matrix(F_spin)

size = 2**14

num_repetitions = size // F_spin.shape[0]

# Create a list of H_spin matrices
H_spin_blocks = [F_spin] * num_repetitions

# Create the block diagonal matrix
large_H_spin_block = la.block_diag(*H_spin_blocks)

# Convert to a sparse matrix
large_H_spin_sparse_csr = sp.csr_matrix(large_H_spin_block)


converged SCF energy = -76.00933224033


(16378, 16378)

#### See the true lowest eigenvalue

In [17]:

# Assuming large_H_spin_sparse_csr is your large sparse Hamiltonian matrix
# Find the smallest eigenvalue
# 'which='SA'' means to find the smallest algebraic eigenvalue
eigenvalues, eigenvectors = eigsh(large_H_spin_sparse_csr, k=1, which='SA')

# Extract the smallest eigenvalue
smallest_eigenvalue = eigenvalues[0]
print("Smallest eigenvalue:", smallest_eigenvalue)

Smallest eigenvalue: -63.73326475196806


In [6]:
# Use the spin Hamiltonian directly without enlarging
H_spin_sparse_csr = sp.csr_matrix(H_spin)

# Find the smallest eigenvalue
eigenvalues, eigenvectors = eigsh(H_spin_sparse_csr, k=1, which='SA')

# Extract the smallest eigenvalue
smallest_eigenvalue = eigenvalues[0]
print("Smallest eigenvalue:", smallest_eigenvalue)


Smallest eigenvalue: -31.88784079412096


#### Check the size of the matrix

In [4]:
large_H_spin_sparse_csr.shape

(16384, 16384)

In [11]:
from pyscf import gto, scf
import numpy as np
import scipy.linalg as la
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh

# Define the molecule
mol = gto.M(atom=[
    ['O', (0.0, 0.0, 0.0)],          # Oxygen at origin
    ['H', (0.0, -0.757, 0.587)],     # Hydrogen 1
    ['H', (0.0, 0.757, 0.587)]       # Hydrogen 2
], basis='6-311G')

# Perform Hartree-Fock Calculation
mf = scf.RHF(mol)
mf.kernel()

# Obtain Molecular Orbitals Coefficients and Integrals
mo_coeff = mf.mo_coeff
h1 = mf.get_hcore(mol)
g2 = mol.intor('int2e', aosym='s1')

# Transform Two-Electron Integrals to MO Basis
g2_mo = np.einsum('pqrs,pi,qj,rk,sl->ijkl', g2, mo_coeff, mo_coeff, mo_coeff, mo_coeff)

# Number of orbitals and occupied orbitals
num_orbitals = mo_coeff.shape[1]
nocc = mol.nelectron // 2  # Assuming a closed-shell molecule

# Compute the Density Matrix
P = 2 * np.dot(mo_coeff[:, :nocc], mo_coeff[:, :nocc].T)

# Construct the Fock Matrix
F = np.copy(h1)
for i in range(num_orbitals):
    for j in range(num_orbitals):
        for k in range(num_orbitals):
            for l in range(num_orbitals):
                F[i, j] += P[k, l] * (2 * g2_mo[i, j, k, l] - g2_mo[i, k, j, l])

# Expand the Fock Matrix to include spin orbitals
num_spin_orbitals = 2 * num_orbitals
F_spin = np.zeros((num_spin_orbitals, num_spin_orbitals))
for i in range(num_orbitals):
    for j in range(num_orbitals):
        F_spin[i, j] = F[i, j]
        F_spin[i + num_orbitals, j + num_orbitals] = F[i, j]

# Convert the Fock Matrix to a sparse matrix
F_spin_sparse = sp.csr_matrix(F_spin)

# Find the smallest eigenvalue
eigenvalues, eigenvectors = eigsh(F_spin_sparse, k=1, which='SA')

# Extract the smallest eigenvalue
smallest_eigenvalue = eigenvalues[0]
print("Smallest eigenvalue:", smallest_eigenvalue)


converged SCF energy = -76.00933224033
Smallest eigenvalue: -63.73326475196809


In [12]:
# Calculate the Electronic Energy
E_electronic = 0.5 * np.sum(P * (h1 + F))

# Add the Nuclear Repulsion Energy
E_nuclear = mol.energy_nuc()

# Total Molecular Energy
E_total = E_electronic + E_nuclear

print("Total Electronic Energy:", E_electronic)
print("Nuclear Repulsion Energy:", E_nuclear)
print("Total Molecular Energy:", E_total)


Total Electronic Energy: -109.6061579514529
Nuclear Repulsion Energy: 9.188258417746113
Total Molecular Energy: -100.41789953370679
